In [34]:
import numpy as np
from scipy.stats import gmean
import cvxopt as opt
from cvxopt import matrix, spmatrix, sparse
from cvxopt.solvers import qp, options
from cvxopt import blas
import pandas as pd

###### Functions

In [35]:
def geo_mean(returns):
    geo = []
    n = len(returns.transpose())
    for i in range(0,n):
        geo = geo + [np.exp(np.log(rets[:,i]+1).mean())-1]
    return geo

## VaR Optimization

##   min     gamma + (1 / [(1 - alpha) * S]) * sum( z_s )
###   s.t.    z_s   >= 0,                 for s = 1, ..., S
###           z_s   >= -r_s' x - gamma,   for s = 1, ..., S  
###           1' x  =  1,
###           mu' x >= R

In [72]:
prices = pd.read_csv('prices.csv',header=None).to_numpy()
rets = prices[1:,:]/prices[0:-1,:] -1
mu = geo_mean(rets)
# Number of Scenarios, Assets
S,n = np.shape(rets)


R = 0.0021
alpha = 0.95

In [49]:
np.shape(mu)

(50,)

In [161]:
lb = np.concatenate((-1*np.ones(n)*np.inf,np.zeros(S),-1*np.ones(1)*np.inf))

In [143]:
ub = np.array(np.shape(lb)[0]*[None])

In [162]:
bounds = np.transpose(np.array((lb,ub)))

In [64]:
A = np.block([[-1*rets,-1*np.eye(S),-1*np.ones((S,1))],[-1*np.array(mu),np.zeros((1,S)),0]])

In [67]:
b = np.block([[np.zeros((S,1))],[np.ones((1,1))*R]])

In [69]:
Aeq = np.block([[np.ones((1,n)), np.zeros((1,S)), np.zeros((1,1))]])

In [71]:
beq = np.block([[1]])

In [73]:
k = (1 / (1 - alpha) * S)

In [76]:
c = np.block([[np.zeros((n,1))],[k*np.ones((S,1))],[np.ones((1,1))]])

In [77]:
np.shape(c)

(261, 1)

In [163]:
import scipy

In [164]:
a = scipy.optimize.linprog(c,A,b,Aeq,beq,bounds)

C:\Users\samga\AppData\Local\Temp\ipykernel_20236\2874196093.py:1: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  a = scipy.optimize.linprog(c,A,b,Aeq,beq,bounds)
C:\Users\samga\AppData\Local\Temp\ipykernel_20236\2874196093.py:1: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  a = scipy.optimize.linprog(c,A,b,Aeq,beq,bounds)
C:\Users\samga\anaconda3\lib\site-packages\scipy\optimize\_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.79747e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)
C:\Users\samga\anaconda3\lib\site-packages\scipy\optimize\_linprog_ip.py:117:

In [169]:
a['x'][0:50]

array([-0.09626304, -0.06666094,  0.21578517, -0.01355142, -0.10743482,
       -0.04466234,  0.26805008,  0.10367581,  0.10138764,  0.00649321,
        0.39160378,  0.37909815, -0.13808316,  0.07346121, -0.25019238,
       -0.57886144,  0.00145727, -0.43736393, -0.41360082,  0.28378487,
        0.01516726,  0.12121183, -0.26888968,  0.25724851,  0.18651047,
        0.14990374,  0.51596554, -0.23859757,  0.05211026,  0.01909319,
       -0.03272722,  0.12768684, -0.22669518, -0.28680111, -0.15651686,
        0.35607074,  0.35772266, -0.31812894, -0.04185119,  0.11155846,
        0.11824785, -0.13192803, -0.05050348,  0.14998272, -0.03454624,
        0.24726307, -0.08585172,  0.39173524, -0.19172901,  0.18423769])

In [154]:
np.shape(np.transpose(np.array((lb,ub))))

(261, 2)